Zunächst die Standardimports der Libraries

In [24]:
import numpy as np
#import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from itertools import islice
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Import der Daten
(erstmal nur die ersten 5 Bilder) (mittlerweile alle)

In [25]:
currentPath = Path().cwd()

targetPath = currentPath.parent / 'data' / 'KGT_noDefect_simplified'
imsNoF = [[str(file), False] for file in targetPath.iterdir() if file.is_file()]

targetPath = currentPath.parent / 'data' / 'KGT_pitting_simplified'
imsF = [[str(file), True] for file in targetPath.iterdir() if file.is_file()]

ims = imsNoF + imsF

del(imsNoF, imsF, currentPath, targetPath)

zur veranschaulichung Umwandlung in Grey Scale

In [26]:
'''
imsGray = []
for file, _ in ims:
    imsGray.append(cv2.cvtColor(file, cv2.COLOR_BGR2GRAY))
'''

'\nimsGray = []\nfor file, _ in ims:\n    imsGray.append(cv2.cvtColor(file, cv2.COLOR_BGR2GRAY))\n'

finden von Kanten mit Sobel

In [27]:
'''
sobelIms = []
for image in imsGray:
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    sobelIms.append(cv2.convertScaleAbs(cv2.magnitude(sobelx,sobely)))
'''

'\nsobelIms = []\nfor image in imsGray:\n    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)\n    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)\n    sobelIms.append(cv2.convertScaleAbs(cv2.magnitude(sobelx,sobely)))\n'

Für den Moment damit bereits dreifache Trainingsdaten. Kanten halte ich für äußerst hilfreich.

In [28]:
'''
cv2.imshow('Color', ims[0][0])
cv2.imshow('gray', imsGray[0][0])
cv2.imshow('sobel', sobelIms[0])

cv2.waitKey(0)
cv2.destroyAllWindows()
'''

"\ncv2.imshow('Color', ims[0][0])\ncv2.imshow('gray', imsGray[0][0])\ncv2.imshow('sobel', sobelIms[0])\n\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [29]:
'''
imData = pd.DataFrame(ims,columns=['image', 'label'])
imData['gray image'] = imsGray
imData['sobel image'] = sobelIms
del(ims, imsGray, sobelIms)
'''

"\nimData = pd.DataFrame(ims,columns=['image', 'label'])\nimData['gray image'] = imsGray\nimData['sobel image'] = sobelIms\ndel(ims, imsGray, sobelIms)\n"

In [30]:
def processImage(path, size=(150,150)):
    img = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    img = cv2.resize(img, size)
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    sobel = cv2.magnitude(sobelx, sobely)
    features = sobel.flatten()
    return features

In [31]:
for i in range(len(ims)):
    processedImage = processImage(ims[i][0])
    if processedImage is not None:
        ims[i][0] = processedImage

In [32]:
images, labels = zip(*ims)
X = np.array(images)
y = np.array(labels)
XTrain, XTest, yTrain, yTest = train_test_split(X,y, stratify=y, test_size=0.3, random_state=42)

In [33]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(XTrain, yTrain)
yPred = clf.predict(XTest)
print(classification_report(yTest,yPred))

              precision    recall  f1-score   support

       False       0.77      0.77      0.77       194
        True       0.82      0.83      0.83       255

    accuracy                           0.80       449
   macro avg       0.80      0.80      0.80       449
weighted avg       0.80      0.80      0.80       449

